In [1]:
#import sys
#!{sys.executable} -m pip install pandas

import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv("attacks.csv", encoding='ISO-8859-1') #in case we need to open in Visual Studio Code, the path should be absolute. df = pd.read_csv("/Users/ale/Proyectos Ironhack/Miniproject_2/attacks.csv", encoding='ISO-8859-1')
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [3]:
print(df.shape)

(25723, 24)


In [4]:
df.info() #to find out what type are the columns and how many cells contain values.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [5]:
df.isnull().sum() #to find out how many cells are null or empty on each column

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

In [6]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [7]:
# String manipulation to rename columns and correct misspellings.
df.rename(columns = {'Sex ': 'Sex','Species ': 'Species'}, inplace = True)
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [8]:
# Drop repeated columns that do not bring any new information.

#Elimino 'Case Number', 'Case Number.1' y 'Case Number.2' porque contienen la misma info que podemos encontrar en 'Date'
#Elimino 'Investigator or source', 'href formula', 'href' y 'original order' porque no considero que su contenido aporte información para mi análisis.
#Elimino 'pdf' ya que sus valores son los nombres de los archivos pero no puedo acceder a ellos.
#Elimino las columnas 'Unnamed:22' y 'Unnamed:23' porque sus valores son 'Nan'.
clean_data = df.drop(columns=['Case Number','Name','Investigator or Source','pdf','href formula','href','Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22','Unnamed: 23'])
clean_data

,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Time,Species
0,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark
1,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN
2,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN
3,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,M,NaN,Minor injury to lower leg,N,NaN,2 m shark
4,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Drop NaN values

clean_data.dropna(thresh=3, inplace = True) #Si una fila contiene almenos 3 valores, se queda. Nos deshacemos de las filas que contienen solamente 1 o 2 valores, por aportar poca información.

In [10]:
print(clean_data.isnull().sum()) #to find out how many cells are null/empty on each column on this dataframe.

Date              0
Year              2
Type              4
Country          50
Area            455
Location        540
Activity        544
Sex             565
Age            2831
Injury           28
Fatal (Y/N)     539
Time           3354
Species        2838
dtype: int64


In [11]:
# Change Year column type, from Float to Integer.
clean_data['Year'].fillna(0,inplace=True)
clean_data['Year'] = clean_data['Year'].astype(int) 
display(clean_data.dtypes)

Date           object
Year            int64
Type           object
Country        object
Area           object
Location       object
Activity       object
Sex            object
Age            object
Injury         object
Fatal (Y/N)    object
Time           object
Species        object
dtype: object

In [12]:
clean_data["Year"].value_counts()

2015    143
2017    136
2016    130
2011    128
0       127
       ... 
1786      1
1758      1
1742      1
1738      1
1554      1
Name: Year, Length: 249, dtype: int64

In [13]:
clean_data["Year"].describe() 
#Para hacer un análisis estadístico de los años de los ataques registrados

count    6302.000000
mean     1926.660743
std       283.160558
min         0.000000
25%      1942.000000
50%      1977.000000
75%      2005.000000
max      2018.000000
Name: Year, dtype: float64

In [14]:
# We will keep rows from 1960 til 2018, and will drop the rest.

clean_data = clean_data.drop(clean_data[clean_data["Year"]<1960].index)
clean_data["Year"].describe() 

count    4033.000000
mean     1994.787007
std        17.960343
min      1960.000000
25%      1981.000000
50%      2000.000000
75%      2010.000000
max      2018.000000
Name: Year, dtype: float64

In [15]:
clean_data["Country"].value_counts() #investigo las columnas para conseguir información relevante

USA                               1756
AUSTRALIA                          704
SOUTH AFRICA                       432
BRAZIL                             103
BAHAMAS                             91
                                  ... 
INDIA                                1
KIRIBATI                             1
NIGERIA                              1
NORWAY                               1
FEDERATED STATES OF MICRONESIA       1
Name: Country, Length: 138, dtype: int64

In [16]:
# Regex in columns where there are "few" unique values (such as 'Type', 'Sex','Fatal (Y/N)', Activity,)
clean_data["Type"].value_counts() 

Unprovoked      3037
Provoked         367
Invalid          343
Boat             111
Boating           92
Sea Disaster      78
Questionable       2
Boatomg            1
Name: Type, dtype: int64

In [17]:
clean_data["Type"]=clean_data["Type"].replace("Boat", "Boating")
clean_data["Type"]=clean_data["Type"].replace("Boatomg", "Boating")

clean_data["Type"].value_counts() 
# another solution:
# clean_data['Type']=np.where(clean_data['Type'].str.startswith('Bo'),"Boating", clean_data['Type'])

Unprovoked      3037
Provoked         367
Invalid          343
Boating          204
Sea Disaster      78
Questionable       2
Name: Type, dtype: int64

In [18]:
# Regex in columns where there are "few" unique values (such as 'Type', 'Sex','Fatal (Y/N)', Activity,)
clean_data["Sex"].unique()

array(['F', 'M', nan, 'M ', 'lli'], dtype=object)

In [19]:
# Regex in columns where there are "few" unique values (such as 'Type', 'Sex','Fatal (Y/N)', Activity,)
clean_data["Sex"]=clean_data["Sex"].replace("M ", "M")
clean_data["Sex"].value_counts() 

M      3182
F       515
lli       1
Name: Sex, dtype: int64

In [20]:
# Regex in columns where there are "few" unique values (such as 'Type', 'Sex','Fatal (Y/N)', Activity,)
clean_data["Fatal (Y/N)"].unique() #

array(['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N'], dtype=object)

In [21]:
# Regex in columns where there are "few" unique values (such as 'Type', 'Sex','Fatal (Y/N)', Activity,)
clean_data["Fatal (Y/N)"]=clean_data["Fatal (Y/N)"].str.replace('M', 'N')
clean_data["Fatal (Y/N)"]=clean_data["Fatal (Y/N)"].str.replace(' N', 'N')
clean_data["Fatal (Y/N)"]=clean_data["Fatal (Y/N)"].str.replace('2017', 'UNKNOWN')

# fill nan values with UNKNOWN
clean_data['Fatal (Y/N)'].fillna('UNKNOWN',inplace=True)

clean_data["Fatal (Y/N)"].unique() #

# OPTIONAL: To remove any value called "2017" within the "Fatal Y/N" Column.
#clean_data = clean_data[clean_data["Fatal (Y/N)"] != "2017"] 

array(['N', 'Y', 'UNKNOWN'], dtype=object)

In [22]:
# String manipulation to rename values within 'Date' column. We would need to do regex to remove the day and year and keep only the month, so we can do some analysis in this area.
clean_data["Date"]=clean_data["Date"].str.replace('\d+', '', regex=True) # set up as true because we want to replace any digits with an empty string.
clean_data["Date"]=clean_data["Date"].str.replace('-', '')
clean_data["Date"]=clean_data["Date"].str.replace(' ', '')
clean_data["Date"]=clean_data["Date"].str.replace('"', '')
clean_data["Date"]=clean_data["Date"].str.replace('Jultohavehappenedontheweekend', '')
clean_data["Date"]=clean_data["Date"].str.replace('LastincidentofinHongKong', '')
clean_data["Date"]=clean_data["Date"].str.replace('Early', '')
clean_data["Date"]=clean_data["Date"].str.replace('Summer', '')
clean_data["Date"]=clean_data["Date"].str.replace('summer', '')
clean_data["Date"]=clean_data["Date"].str.replace('Mid', '')
clean_data["Date"]=clean_data["Date"].str.replace('Reported', '')
clean_data["Date"]=clean_data["Date"].str.replace('Late', '')
clean_data["Date"]=clean_data["Date"].str.replace('June', 'Jun')
clean_data["Date"]=clean_data["Date"].str.replace('orMay', 'May')
clean_data["Date"]=clean_data["Date"].str.replace('December', 'Dec')
clean_data["Date"]=clean_data["Date"].str.replace('`', '')
clean_data["Date"]=clean_data["Date"].str.replace('November', 'Nov')
clean_data["Date"]=clean_data["Date"].str.replace('.', '', regex=False) #we only want to replace that specific character(dot)
clean_data["Date"]=clean_data["Date"].str.replace('July', 'Jul')
clean_data["Date"]=clean_data["Date"].str.replace('Jula', 'Jul')
clean_data["Date"]=clean_data["Date"].str.replace('Julb', 'Jul')
clean_data["Date"]=clean_data["Date"].str.replace('s', '')
clean_data["Date"]=clean_data["Date"].str.replace('Ca', '')
clean_data["Date"]=clean_data["Date"].str.replace('MayJun', '')
clean_data["Date"]=clean_data["Date"].str.replace('Fall', '')
clean_data["Date"]=clean_data["Date"].str.replace('of', '')
clean_data["Date"]=clean_data["Date"].str.replace('JulormidJul', '')
clean_data["Date"]=clean_data["Date"].str.replace('Winter', '')
clean_data["Date"]=clean_data["Date"].str.replace('JanJun', '')
clean_data["Date"]=clean_data["Date"].str.replace('BetweenMay&Nov', '')
clean_data["Date"]=clean_data["Date"].str.replace(r'\bAp\b', 'Apr', regex=True)
clean_data["Date"]=clean_data["Date"].str.replace('Jultohavehappenedontheweekend', '')
clean_data["Date"]=clean_data["Date"].str.replace(r'^\s*$', 'UNKNOWN', regex=True) # to replace an empty string with UNKNOWN.

clean_data["Date"].value_counts()

Jul        432
Aug        411
Sep        400
Jun        361
Apr        328
Jan        328
Oct        327
Mar        276
May        275
Dec        270
Nov        261
Feb        256
UNKNOWN    108
Name: Date, dtype: int64

In [23]:
# String manipulation to rename "Date" column.
clean_data.rename(columns={'Date':'Month'}, inplace = True)

In [24]:
clean_data["Month"].unique()

array(['Jun', 'May', 'Apr', 'Mar', 'Feb', 'Jan', 'Dec', 'Nov', 'Oct',
       'Sep', 'Aug', 'Jul', 'UNKNOWN'], dtype=object)

In [25]:
clean_data["Activity"].value_counts()

Surfing                                      958
Swimming                                     495
Spearfishing                                 283
Fishing                                      220
Wading                                       124
                                            ... 
Diving into water                              1
Swimming or paddle boarding                    1
Floating on his back in an inner tube          1
Catching sharks under government contract      1
Fishing from surfski                           1
Name: Activity, Length: 810, dtype: int64

In [26]:
#para agrupar actividades similares a fishing (spearfishing, etc)
clean_data["Activity"].replace(to_replace = 
r'[a-zA-Z]*\s*fishing\s*[a-zA-Z]*', 
value = "Fishing", inplace = True, regex = True)

# ANOTHER OPTION?
#clean_data["Activity"] = clean_data["Activity"].replace(r'[a-zA-Z]*\s*fishing\s*[a-zA-Z]*', 'Fishing')

In [27]:
#para agrupar actividades similares a surfing (kite surfing, windsurfing, etc)
clean_data["Activity"].replace(to_replace = 
r'[a-zA-Z]*\s*surfing\s*[a-zA-Z]*', 
value = "Surfing", inplace = True, regex = True)


In [28]:
clean_data["Fatal (Y/N)"]=clean_data["Fatal (Y/N)"].str.replace('nan', 'UNKNOWN')
clean_data["Fatal (Y/N)"].unique() #

array(['N', 'Y', 'UNKNOWN'], dtype=object)

In [29]:
clean_data["Activity"].value_counts() 

Surfing                                                             1019
Fishing                                                              545
Swimming                                                             495
Wading                                                               124
Snorkeling                                                            88
                                                                    ... 
Jumping in the waves                                                   1
Petting a shark                                                        1
Fishing with hand line tied to wrist & was pulled into the water       1
Adrift after ditching plane in the sea                                 1
Scuba diving / culling lionfish                                        1
Name: Activity, Length: 781, dtype: int64

In [30]:
clean_data

,Month,Year,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Time,Species
0,Jun,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark
1,Jun,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN
2,Jun,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN
3,Jun,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,M,NaN,Minor injury to lower leg,N,NaN,2 m shark
4,Jun,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4029,UNKNOWN,1960,Unprovoked,FIJI,Northwest of Viti Levu,"Covull Reef, near Lautoka",Fishing,M,NaN,"Right arm abraded, circular piece of tissue re...",N,NaN,NaN
4030,UNKNOWN,1960,Unprovoked,PAPUA NEW GUINEA,Madang,NaN,"Fishing, removing fish from spear",M,16,"FATAL, foot severed, hip bitten",Y,NaN,NaN
4031,UNKNOWN,1960,Unprovoked,PAPUA NEW GUINEA,West New Britain Province,"West Nakanai, Talasea",Fishing,M,20,Slight lacerations to leg,N,NaN,NaN
4032,UNKNOWN,1960,Unprovoked,MOZAMBIQUE,Gaza,Xai Xai,Swimming,F,5,FATAL,Y,NaN,NaN


In [31]:
# We save the final dataset that we will use to do the analysis.
clean_data.to_csv('output/clean_data.csv')
